In [3]:
cd '/home/gdolle/code/G-Dolle/DIVVY_BIKE'

/home/gdolle/code/G-Dolle/DIVVY_BIKE


In [4]:
import os
import pandas as pd
import numpy as np
import math
from datetime import date, time, datetime

from ml_logic.data_import import get_weather_data, get_divvy_data
from ml_logic.cleaning import weather_cleaning, station_stats, cleaning_divvy, merge_divvy_weather, features_target
from ml_logic.preprocessor import transform_time_features, preprocess_features, target_process

from ml_logic.main import preprocess


In [6]:
quarter= os.environ.get("DIVVY_QUARTER")
year= os.environ.get("DIVVY_YEAR")

In [9]:
df = get_divvy_data(year,quarter)
df.head(2)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,47EC0A7F82E65D52,classic_bike,2022-03-21 13:45:01,2022-03-21 13:51:18,Wabash Ave & Wacker Pl,TA1307000131,Kingsbury St & Kinzie St,KA1503000043,41.886875,-87.626030,41.889177,-87.638506,member
1,8494861979B0F477,electric_bike,2022-03-16 09:37:16,2022-03-16 09:43:34,Michigan Ave & Oak St,13042,Orleans St & Chestnut St (NEXT Apts),620,41.900998,-87.623752,41.898203,-87.637536,member


In [10]:
df['started_at']=pd.to_datetime(df['started_at'])
df['ended_at']=pd.to_datetime(df['ended_at'])
df['hourly_data_started'] = df.started_at.dt.round('60min')
df['hourly_data_ended'] = df.ended_at.dt.round('60min')

In [26]:
df_departures=df[["started_at",
                 "start_station_name",
                 "start_station_id",
                 "hourly_data_started"]]

df_departures=df_departures.rename(columns={'hourly_data_started':'hourly_data'})

df_departures.head(2)

,started_at,start_station_name,start_station_id,hourly_data
0,2022-03-21 13:45:01,Wabash Ave & Wacker Pl,TA1307000131,2022-03-21 14:00:00
1,2022-03-16 09:37:16,Michigan Ave & Oak St,13042,2022-03-16 10:00:00


In [27]:
df_departures=df_departures.groupby(by=["start_station_name",
                                        "start_station_id",
                                        'hourly_data']).count().reset_index()
df_departures.head(10)

,start_station_name,start_station_id,hourly_data,started_at
0,2112 W Peterson Ave,KA1504000155,2022-01-01 21:00:00,1
1,2112 W Peterson Ave,KA1504000155,2022-01-03 18:00:00,1
2,2112 W Peterson Ave,KA1504000155,2022-01-04 11:00:00,1
3,2112 W Peterson Ave,KA1504000155,2022-01-04 18:00:00,1
4,2112 W Peterson Ave,KA1504000155,2022-01-10 09:00:00,1
5,2112 W Peterson Ave,KA1504000155,2022-01-10 10:00:00,1
6,2112 W Peterson Ave,KA1504000155,2022-01-12 16:00:00,1
7,2112 W Peterson Ave,KA1504000155,2022-01-12 21:00:00,1
8,2112 W Peterson Ave,KA1504000155,2022-01-13 16:00:00,1
9,2112 W Peterson Ave,KA1504000155,2022-01-13 18:00:00,1


In [20]:
df_departures[df_departures["start_station_name"]=="California Ave & Cortez St"]

,start_station_name,start_station_id,hourly_data,started_at
22794,California Ave & Cortez St,17660,2022-01-01 00:00:00,1
22795,California Ave & Cortez St,17660,2022-01-02 11:00:00,1
22796,California Ave & Cortez St,17660,2022-01-02 17:00:00,1
22797,California Ave & Cortez St,17660,2022-01-02 19:00:00,1
22798,California Ave & Cortez St,17660,2022-01-03 13:00:00,1
...,...,...,...,...
23230,California Ave & Cortez St,17660,2022-03-31 07:00:00,1
23231,California Ave & Cortez St,17660,2022-03-31 08:00:00,1
23232,California Ave & Cortez St,17660,2022-03-31 09:00:00,1
23233,California Ave & Cortez St,17660,2022-03-31 15:00:00,1


In [21]:
df_departures.started_at.value_counts()

1    235828
Name: started_at, dtype: int64

In [ ]:
def cleaning_divvy_gen(df,station_name):

    df['started_at']=pd.to_datetime(df['started_at'])
    df['ended_at']=pd.to_datetime(df['ended_at'])
    df['hourly_data_started'] = df.started_at.dt.round('60min')
    df['hourly_data_ended'] = df.ended_at.dt.round('60min')

    # Departures per station
    df_departures=df[df['start_station_name']==station_name]
    df_departures=df_departures[['started_at','hourly_data_started']]
    df_departures=df_departures.rename(columns={'hourly_data_started':'hourly_data'})
    df_departures=df_departures.groupby(by='hourly_data').count()


    # Arrivals per station
    df_arrivals=df[df['end_station_name']==station_name]
    df_arrivals=df_arrivals[['ended_at','hourly_data_ended']]
    df_arrivals=df_arrivals.rename(columns={'hourly_data_ended':'hourly_data'})
    df_arrivals=df_arrivals.groupby(by='hourly_data').count()

    # Merge departures and arrivals to get a ratio
    merge_ratio=pd.merge(
    df_departures,
    df_arrivals,
    how="outer",
    on='hourly_data')

    merge_ratio.rename(columns={"started_at":"nb_departures","ended_at":"nb_arrivals"}, inplace=True)

    merge_ratio["nb_departures"] = merge_ratio["nb_departures"].replace(np.nan, 0)
    merge_ratio["nb_arrivals"] = merge_ratio["nb_arrivals"].replace(np.nan, 0)

    merge_ratio['ratio']=merge_ratio['nb_departures']/merge_ratio['nb_arrivals']

    # Return the merged dataset
    return merge_ratio